# With Wandb pipline

### setup environment

In [58]:
%pip install transformers datasets
%pip install wandb
%pip install torch
%pip install torchmetrics
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### import

In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm.auto import tqdm

In [60]:
import wandb

# %wandb login
wandb.login(key="45c157220ede05498be4a005ad27ca6ab56bcdd1")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [61]:
# Set Ramdom
# random.seed(42)

## Data Preprocessing

In [62]:
# import pandas as pd
# import random

# from CangjieKeyMap import CangjieKeyMap


# def cangjie_mixer():
#     CHINESE_NEWS_TXT_PATH = ".\dataset\\chinese_news.txt"
#     CHINESE_NEWS_CANGJIE_MIX_PATH = ".\dataset\chinese_news_cangjie_mix.txt"

#     cangjie_key_map = CangjieKeyMap()

#     with open(CHINESE_NEWS_TXT_PATH, "r", encoding="utf-8") as news_file:
#         with open(CHINESE_NEWS_CANGJIE_MIX_PATH, "w", encoding="utf-8") as news_mix_file:
#             for line in news_file:
#                 line = line.strip()
#                 cangjie_line = cangjie_key_map.convert_to_cangjie_key(line)
#                 news_mix_file.write(cangjie_line + "\t" + line + "\n")

# cangjie_mixer()

## Training

### Configs

In [63]:
# Gobal variables
PROGJECT_NAME = 'WIDM-MuIME'
DATASET_ROOT_PATH = "./datasets"
MODEL_SAVE_PATH = "./"
CHINESE_NEWS_CANGJIE_MIX_DATASETS_PATH = "./datasets/chinese_news_cangjie_mix.txt"
SKIP_WANDB_LOG = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [64]:
config = dict(
    epochs = 10,
    learning_rate = 3e-4,
    batch_size = 16,
    optimizer = "Adam",
    sequence_size = 128,
    embedding_size = 768,
    example_size = 500,
    # classes=10,
    # kernels=[16, 32],
    # dataset="MNIST",
    device=device,
    architecture = "Transformer",
)

## Pipline

In [65]:
def tensorlog(name:str, x:torch.Tensor) -> None:
    print("{}: {}".format(name, x))
    print("{} shape: {}".format(name, x.shape))

### Model

In [66]:
from transformers import AutoModel
model_name = "bert-base-chinese"
bert_embedding = AutoModel.from_pretrained(model_name).get_input_embeddings()

class Cangjie2ChineseTransformer(nn.Module): # Todo: write size
    def __init__(
            self,
            embedding_size=768, # fix
            src_vocab_size=54,
            trg_vocab_size=21128,
            src_pad_idx=0,
            num_heads=8,
            num_encoder_layers=3,
            num_decoder_layers=3,
            dim_feedforward=2048,
            dropout=0.1,
            sequence_size=128,
            device=None # fix this
        ) -> None:
        """
        N -> Batch size,
        S -> sequence len,
        E -> embedding dim
        C -> class (tokens)
        """


        super().__init__()
        self.model_type = 'Transformer'
        self.embedding_size = embedding_size # E
        self.sequence_size = sequence_size # S
        self.device=device

        #
        self.src_word_embedding = nn.Embedding(src_vocab_size, embedding_size, padding_idx=src_pad_idx).to(self.device)
        self.src_position_embedding = nn.Embedding(sequence_size, embedding_size, padding_idx=src_pad_idx).to(self.device)
        # self.trg_word_embedding = nn.Embedding(trg_vocab_size, embedding_size, padding_idx=src_pad_idx)
        self.trg_word_embedding = bert_embedding.to(self.device)
        self.trg_position_embedding = nn.Embedding(sequence_size, embedding_size, padding_idx=src_pad_idx).to(self.device)

        self.transformer = nn.Transformer(
            d_model=embedding_size,
            nhead=num_heads,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.src_pad_idx = src_pad_idx

    def make_src_mask(self, src):
        src_mask = src == self.src_pad_idx

        # (N, src_len)
        return src_mask.to(self.device)

    def forward(self, src:torch.Tensor, trg:torch.Tensor) -> torch.Tensor:
        N, src_seq_length = src.shape
        N, trg_seq_length = trg.shape

        src_positions = (  # (N, S, E)
            torch.arange(0, src_seq_length)
            .expand(N, src_seq_length)
            .to(self.device)
        )
        trg_positions = (   # (N, S, E)
            torch.arange(0, trg_seq_length)
            .expand(N, trg_seq_length)
            .to(self.device)
        )


        embed_src = self.dropout(  # (N, S, E)
            (self.src_word_embedding(src).to(self.device) + self.src_position_embedding(src_positions).to(self.device) )
        )
        embed_trg = self.dropout(  # (N, S, E)
            (self.trg_word_embedding(trg).to(self.device) + self.trg_position_embedding(trg_positions).to(self.device) )
        )
		# <pad> 加入 mask
        src_padding_mask = self.make_src_mask(src) # 可能會有問題?  # (N, S)
        # print("src_padding_mask", src_padding_mask.shape)


        # 訓練的時候要加入 mask 不然會看到後面的答案
        # decoder 用的 mask call sequence mask
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(  # (S, S)
            self.device
        )
        # print("trg_mask: ", trg_mask.shape)

        out = self.transformer( # (N, S, E)
            embed_src,
            embed_trg,
            src_key_padding_mask=src_padding_mask,
            tgt_mask=trg_mask,
        )
        out = self.fc_out(out) # (N, S, C)

        return out


# model = Cangjie2ChineseTransformer()


### Dataset

In [67]:
from torch.utils.data import Dataset

class MyDataSet(Dataset):
    def __init__(self, X, Y, transform=None):
        self.data = X
        self.labels = Y

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.labels[index]

### train

In [68]:
import sys

def train(model, train_loader, val_loader, criterion, optimizer, config):
    print("start training...")
    wandb.watch(model, criterion, log="all", log_freq=10)


    model.to(config["device"])
    
    total_batches = len(train_loader) * config.epochs
    total_batch = len(train_loader)
    example_ct = 0  # number of examples seen

    best_val_acc = 0
    for epoch in tqdm(range(config.epochs)):

        # ==================== Train =====================
        model.train()
        train_loss, train_acc = 0.0, 0.0
        for t_batch, (inputs, labels) in enumerate(train_loader):
            batch_loss, correct_batch_token, total_batch_token = train_batch(inputs, labels, model, optimizer, criterion)
            batch_acc = correct_batch_token / total_batch_token
            example_ct +=  len(inputs)
            train_loss += batch_loss

            train_log(epoch, t_batch, total_batch, batch_loss, batch_acc, example_ct)


        # calculate train_acc
        print('\nTrain | Loss: {:.5f} Acc: {:.3f}%'.format(train_loss, train_acc))



        # ==================== Valid =====================
        model.eval()
        val_loss, val_acc = 0, -404
        total_correct_token, total_token = 0, 0
        with torch.no_grad():
            for t_val_batch, (inputs, labels) in enumerate(val_loader):
                val_batch_loss, correct_batch_token, total_batch_token = val_batch(inputs, labels, model, criterion)
                val_batch_acc = correct_batch_token / total_batch_token

                val_loss += val_batch_loss
                total_correct_token += correct_batch_token
                total_token += total_batch_token

        val_acc = total_correct_token / total_token
        val_log(epoch, val_loss, val_acc, example_ct)


        # todo: if val > train safe model
        # save with wandb.save
        if val_acc > best_val_acc:
            print("Saving model with acc {}".format(val_acc))
            torch.save(model.state_dict(), MODEL_SAVE_PATH + '/model.pth')


def train_batch(inputs, labels, model, optimizer, criterion):
    inputs, labels = inputs.to(device), labels.to(device)

    # Forward pass ➡
    outputs = model(inputs, labels) # (N, S, C)
    
    
    
    # matrix = torch.eye(21128).to(device)
    # embed_labels = matrix[labels].permute(0, 2, 1).to(device) # (N, S) -> (N, C, S)
    loss = criterion(outputs.permute(0, 2, 1), labels)


    reduced_outputs = torch.argmax(outputs, dim=-1) # (N, S)
    print("reduced_outputs: ", reduced_outputs)
    print("labels: ", labels)
    acc = (reduced_outputs.flatten(0) == labels.flatten(0)).sum() / labels.numel()


    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1) # avoid exploding gradient

    # Step with optimizer
    optimizer.step()

    return loss.item(), (reduced_outputs.flatten(0) == labels.flatten(0)).sum(), labels.numel()


def val_batch(inputs, labels, model, criterion):
    inputs, labels = inputs.to(device), labels.to(device)

    outputs = model(inputs, labels)
    
    # .permute(0, 2, 1)  # (N, S, C) -> (N, C, S)
    # matrix = torch.eye(21128).to(device)
    # embed_labels = matrix[labels].permute(0, 2, 1)  # (N, S) -> (N, C, S)
    loss = criterion(outputs.permute(0, 2, 1), labels)

    reduced_outputs = torch.argmax(outputs, dim=-1) # (N, S)
    acc = (reduced_outputs.flatten(0) == labels.flatten(0)).sum() / labels.numel()


    return loss.item(), (reduced_outputs.flatten(0) == labels.flatten(0)).sum(), labels.numel()

def train_log(epoch:int, t_batch:int, total_batch:int, batch_loss:float, batch_acc:float, example_ct:int):
    wandb.log({"train_loss": batch_loss, "train_acc": batch_acc, "Epoach": epoch+1}, step=example_ct)
    print("\r[ Epoch {}: {}/{} ] Batch loss: {:.3f} Batch acc: {:.3f}%".format(epoch+1, t_batch+1, total_batch, batch_loss, batch_acc*100), end="")


def val_log(epoch:int, val_loss:int, val_acc:int, example_ct:int):
    wandb.log({"val_loss": val_loss, "val_acc": val_acc}, step=example_ct)
    print("Valid | Loss: {:.5f} Acc: {:.3f}% ".format(val_loss, val_acc*100))

#### Get Dataset

In [69]:
import torch
from transformers import AutoTokenizer, AutoModel

SEQUENCE_SIZE = config["sequence_size"]
EMBEDDING_SIZE = config["embedding_size"]
EXAMPLE_SIZE = config["example_size"]
model_name = "bert-base-chinese"
bert_tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_embedding_model = AutoModel.from_pretrained(model_name).get_input_embeddings()


def load_training_dataset():
    print("loading datasets...")
    dataset_path = CHINESE_NEWS_CANGJIE_MIX_DATASETS_PATH

    X, Y = [], []
    with open(dataset_path, "r", encoding="utf-8") as dataset:
        lines = dataset.readlines()


    class KeystrokeTokenizer(): # todo: make it class method? or make it tokenlist class?
        def __init__(self):
            self.key_labels = ["PAD", "`", "1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "-", "=", "q", "w", "e", "r", "t", "y", "u", "i", "o", "p", "[", "]", "\\",
                               "a", "s", "d", "f", "g", "h", "j", "k", "l", ";", "'", "shift", "z", "x", "c", "v", "b", "n", "m", ",", ".", "/", "ctrl", " ",
                               "<SOS>", "<EOS>", "<UNK>"
            ]

        def tokenizer(self, in_str:str) -> list: # list with token
            token_list = []
            token_list.append("<SOS>")
            for char in in_str:
                if char in self.key_labels:
                    token_list.append(char)
                else:
                    token_list.append("<UNK>")
            token_list.append("<EOS>")
            return token_list

        def token_to_ids(self, token_list:list[str]) -> list[int]:
            id_list = []
            for token in token_list:
                assert token in self.key_labels, "Error: can not convert token not on list"
                id_list.append(self.key_labels.index(token))
            return id_list

    def pad(token_id_list:list[int]) -> list[int]:
        if len(token_id_list) > SEQUENCE_SIZE:
            token_id_list = token_id_list[:SEQUENCE_SIZE]
        else:
            pad_len = SEQUENCE_SIZE - len(token_id_list)
            token_id_list += [0] * pad_len
        return token_id_list


    myTokenizer = KeystrokeTokenizer()
    for i, line in enumerate(lines):
        x_str, y_str = line.strip().split("\t")
        x_tokens = myTokenizer.tokenizer(x_str)
        x_ids = myTokenizer.token_to_ids(x_tokens)
        y_ids = bert_tokenizer(y_str).input_ids
        x_ids = pad(x_ids)
        y_ids = pad(y_ids)

        # print("x:", len(x_ids))
        # print("y:", len(y_ids))
        # print(x_ids)
        # print(y_ids)
        x_ids = torch.tensor(x_ids)
        y_ids = torch.tensor(y_ids)

        X.append(x_ids)
        Y.append(y_ids)
        if i > EXAMPLE_SIZE:
            break

    print("Full Datasets:")
    print("X:", len(X), "Y:", len(Y))
    return MyDataSet(X, Y)

load_training_dataset()

loading datasets...
Full Datasets:
X: 502 Y: 502


In [70]:
import platform

def get_dataset(slice_rate=0.8):
    print("preparing datasets...")
    full_dataset = load_training_dataset()

    at = int(len(full_dataset) * slice_rate)
    train_dataset = torch.utils.data.Subset(full_dataset, indices=range(0, at))
    val_dataset = torch.utils.data.Subset(full_dataset, indices=range(at,))

    return train_dataset, val_dataset


def make_loader(dataset, batch_size):
    NUM_OF_WORKERS = 1 if platform.system() == "Windows" else 2

    loader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         pin_memory=True,
                                         num_workers=NUM_OF_WORKERS,
                                         drop_last=True)
    return loader

#### Make

Define model, criterion, and optimizer.

In [71]:
def make(config):
    # Make the data
    train, val = get_dataset()
    print("Training set size: {}, Validation set size: {}".format(train.__len__(), val.__len__()))

    train_loader = make_loader(train, batch_size=config.batch_size)
    val_loader = make_loader(val, batch_size=config.batch_size)

    # Model
    model = Cangjie2ChineseTransformer(device=device)


    # Make the loss and optimizer
    criterion = nn.CrossEntropyLoss()  # Todo: ignore_index=0 B: pad index is 0
    optimizer = torch.optim.Adam(
        model.parameters(), lr=config.learning_rate)
    # todo: lr sheulare

    return model, train_loader, val_loader, criterion, optimizer

In [72]:
def log_model_info(model) -> None:
    print(model)
    total_parameters = sum(p.numel() for p in model.parameters())
    trainable_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print('\nstart training, parameter total: {}, trainable: {}\n'.format(total_parameters, trainable_parameters))
    wandb.config.update({"Total parameter": total_parameters, "Trainable parameters:": trainable_parameters})

#### Pipline

In [73]:
def model_pipeline(hyperparameters):
    with wandb.init(project=PROGJECT_NAME, config=hyperparameters): # todo: as run
      config = wandb.config

      # make the model, data, and optimization
      model, train_loader, val_loader, criterion, optimizer = make(config)
      log_model_info(model)

      # Training
      train(model, train_loader, val_loader, criterion, optimizer, config)

      # Testing
      # test(model, test_loader)

    return model

# Run

In [74]:
import gc


gc.collect()

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

torch.cuda.empty_cache()
model = model_pipeline(config)

preparing datasets...
loading datasets...
Full Datasets:
X: 502 Y: 502
Training set size: 401, Validation set size: 401
Cangjie2ChineseTransformer(
  (src_word_embedding): Embedding(54, 768, padding_idx=0)
  (src_position_embedding): Embedding(128, 768, padding_idx=0)
  (trg_word_embedding): Embedding(21128, 768, padding_idx=0)
  (trg_position_embedding): Embedding(128, 768, padding_idx=0)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-2): 3 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (linear1): Linear(in_features=768, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=768, bias=True)
          (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((768,), ep

  0%|          | 0/10 [00:00<?, ?it/s]

reduced_outputs:  tensor([[ 9120,  2668, 16381,  ..., 15059,  5898, 15846],
        [ 9158,  3383, 18871,  ...,  8529, 18871, 19160],
        [ 5800,  4553, 19863,  ..., 15059,  3477, 13047],
        ...,
        [20728, 11836, 18811,  ..., 15059, 14060, 18613],
        [19571,  4553,  9346,  ..., 15059, 14060, 14551],
        [ 6185,  9282, 10639,  ...,  3105,  7920,  5413]], device='cuda:0')
labels:  tensor([[ 101, 7591, 6268,  ...,    0,    0,    0],
        [ 101,  523, 3292,  ...,    0,    0,    0],
        [ 101, 1963,  862,  ...,    0,    0,    0],
        ...,
        [ 101, 1217, 1751,  ...,    0,    0,    0],
        [ 101, 5401, 1751,  ...,    0,    0,    0],
        [ 101, 2923, 6716,  ...,    0,    0,    0]], device='cuda:0')
[ Epoch 1: 1/25 ] Batch loss: 10.265 Batch acc: 0.000%reduced_outputs:  tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/activation.py:1160: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343967769/work/aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._native_multi_head_attention(


Valid | Loss: 36.26875 Acc: 81.355% 
Saving model with acc 0.8135546445846558


...

reduced_outputs:  tensor([[ 101, 1751,  704,  ...,    0,    0,    0],
        [ 101, 1751, 1751,  ...,    0,    0,    0],
        [ 101, 1751, 1751,  ...,    0,    0,    0],
        ...,
        [ 101, 1751, 1751,  ...,    0,    0,    0],
        [ 101, 1751, 1751,  ...,    0,    0,    0],
        [ 101, 1751, 1751,  ...,    0,    0,    0]], device='cuda:0')
labels:  tensor([[ 101,  872, 4638,  ...,    0,    0,    0],
        [ 101, 5739, 1751,  ...,    0,    0,    0],
        [ 101, 1751, 4634,  ...,    0,    0,    0],
        ...,
        [ 101,  704, 1751,  ...,    0,    0,    0],
        [ 101, 1963,  862,  ...,    0,    0,    0],
        [ 101, 2255, 2797,  ...,    0,    0,    0]], device='cuda:0')
[ Epoch 9: 1/25 ] Batch loss: 1.233 Batch acc: 81.787%reduced_outputs:  tensor([[ 101, 1751, 1751,  ...,    0,    0,    0],
        [ 101, 1751, 1751,  ...,    0,    0,    0],
        [ 101,  704, 1751,  ...,    0,    0,    0],
        ...,
        [ 101,  704, 1751,  ...,    0,    0,  

 90%|█████████ | 9/10 [01:49<00:11, 11.62s/it]

reduced_outputs:  tensor([[ 101,  704, 8038,  ...,    0,    0,    0],
        [ 101,  704, 1751,  ...,    0,    0,    0],
        [ 101,  101,  704,  ...,    0,    0,    0],
        ...,
        [ 101, 1751, 1751,  ...,    0,    0,    0],
        [ 101,  517,  704,  ...,    0,    0,    0],
        [ 101, 1751,  704,  ...,    0,    0,    0]], device='cuda:0')
labels:  tensor([[ 101, 2537,  519,  ...,    0,    0,    0],
        [ 101, 5918, 5244,  ...,    0,    0,    0],
        [ 101, 2548, 6295,  ...,    0,    0,    0],
        ...,
        [ 101,  704, 6356,  ...,    0,    0,    0],
        [ 101,  523, 6213,  ...,    0,    0,    0],
        [ 101,  523, 6213,  ...,    0,    0,    0]], device='cuda:0')
[ Epoch 10: 1/25 ] Batch loss: 1.105 Batch acc: 83.398%reduced_outputs:  tensor([[ 101,  101, 1751,  ...,    0,    0,    0],
        [ 101, 1751, 1751,  ...,    0,    0,    0],
        [ 101,  101, 1751,  ...,    0,    0,    0],
        ...,
        [ 101, 1751,  517,  ...,    0,    0, 

100%|██████████| 10/10 [02:01<00:00, 12.13s/it]


Epoach,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
train_acc,▄▅▃▄▄▆▄▃▂▄▃▇▇▂▃▄▅▃▂▃▄▄▁▇▁▇▆▆▅▂▃▄▅█▄▅▃▃▅▄
train_loss,█▆▅▅▃▂▃▃▃▂▃▂▁▃▃▃▂▃▃▃▂▂▃▂▃▁▂▂▂▃▃▂▂▁▂▂▂▂▂▂
val_acc,▂▁▆▆▇▇▇▇▇█
val_loss,█▆▃▄▃▂▂▁▁▁
Epoach,10
train_acc,0.81152
train_loss,1.26753
val_acc,0.82916
val_loss,28.35789


#### todos

- [ ] use pip install -r requirements.txt / pip freeze
- [ ] set val > train save model
- [ ] training checkpoint
- [ ] blue score
- [ ] use torch text
- [ ] lr sheduler
- [ ] vscode ssh to colab

### 幾個可思考的方向

- 負的 Entropy Loss?
- 重設Embedding
- 設 start token

#### IDEA
- [ ] 不同的斷詞方式 (斷成不同語言)

In [75]:
# from torchmetrics.classification import MultilabelAccuracy, MulticlassAccuracy
# from torch import tensor
# import numpy as np

# target = tensor([[[1, 0], [0, 1], [0, 0]], [[1, 0], [1, 0], [1, 0]], [[1, 0], [1, 0], [1, 0]], [[1, 0], [1, 0], [1, 0]]])
# preds = tensor(
#      [
#          [[1, 3], [1, 0], [1, 1]],
#          [[0, 0.04], [0.86, 0.780], [0.45, 0.37]],
#          [[0, 0.04], [0.86, 0.780], [0.45, 0.37]],
#          [[0, 1], [1, 1], [1, 1]],
#      ])

# target = tensor([[1, 1], [2, 1]])

# preds = tensor([[[0., 0.], [10., 0.], [0., 0.]],
#                 [[0., 10.], [0., 200.], [10., 0.]],
#                 ])

# print("target:", target.numel())
# print("preds:", preds.shape)

# reduced_prediction_tensor = np.argmax(preds, axis=1)
# print(reduced_prediction_tensor)
# accuracy = len()/(reduced_prediction_tensor.flatten(0) == target).sum()


# print(accuracy)

